In [18]:
from transformers import pipeline

import os

import json
from PIL import Image
import csv

from PIL import Image
pipe = pipeline("visual-question-answering", model="MariaK/vilt_finetuned_200")

/Users/sudhanshu/Desktop/UMASS_COURSES_SEMESTERS/SEM_2/NLP/VITLLMs/vqa/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [30]:

image = Image.open("/Users/sudhanshu/Desktop/UMASS_COURSES_SEMESTERS/SEM_2/NLP/VITLLMs/src/dataset/final_eval/22090.jpg")
question = "Are there fruits in the plate?"
print(question)
a=[]
a.append(pipe(image, question, top_k=1))




Are there fruits in the plate?


Generating CSV file for getting results of segmented images on VILT

In [25]:


# Replace 'pipe' with the actual function you are using to process images and questions

# Path to the directory containing images
image_directory = "/Users/sudhanshu/Desktop/UMASS_COURSES_SEMESTERS/SEM_2/NLP/VITLLMs/src/segmentedImage"
fullDataSet = "/Users/sudhanshu/Desktop/UMASS_COURSES_SEMESTERS/SEM_2/NLP/VITLLMs/src/dataset/combined_data1.json"
# Path to the JSON file containing questions
json_file_path = "/Users/sudhanshu/Desktop/UMASS_COURSES_SEMESTERS/SEM_2/NLP/VITLLMs/src/dataset/question_image_mapping.json"
json_fileId_path="/Users/sudhanshu/Desktop/UMASS_COURSES_SEMESTERS/SEM_2/NLP/VITLLMs/src/dataset/questionId_image_mapping.json"
# Load questions from the JSON file
with open(json_file_path, 'r') as json_file:
    questions = json.load(json_file)

with open(json_fileId_path, 'r') as json_file:
    questionsIds = json.load(json_file)


with open(fullDataSet, 'r') as json_file:
    data = json.load(json_file)

def get_answer_details(question_id, data):
    for annotation in data['annotations']:
        if annotation['question_id'] == question_id:
            return annotation['answer_type'], annotation['multiple_choice_answer']
    return None, None
# Extract annotations
annotations = data.get('annotations', [])

# List to store results
results = []
imageId =[]
questionsStored=[]
answerType= []
expectedAnswer=[]
# Iterate through each file in the image directory
for image_name in os.listdir(image_directory):
    if image_name.endswith(('.png', '.jpg', '.jpeg')):  # Ensure the file is an image
        image_path = os.path.join(image_directory, image_name)
        
        # Open the image
        image = Image.open(image_path)
        image_id = image_name.split('_')[0]
        imageId.append(image_id)
        question_id =questionsIds.get(image_name)
        answer_type, multiple_choice_answer = get_answer_details(question_id, data)
        answerType.append(answer_type)
        expectedAnswer.append(multiple_choice_answer)
        
        # Get the question for the current image from the JSON file
        question = questions.get(image_name)
        questionsStored.append(question)
        # Process the image with the question
        result = pipe(image, question, top_k=1)
       
        # Append the result to the results list
        results.append(result[0]['answer'])

# Print the results
# for res in results:
#     print(res)


In [26]:

csv_file_path = '/Users/sudhanshu/Desktop/UMASS_COURSES_SEMESTERS/SEM_2/NLP/VITLLMs/src/SegmentedImage_Baseline_VIT.csv'

# Write the data to a CSV file
with open(csv_file_path, mode='w', newline='') as file:
    writer = csv.writer(file)
    
    # Write the header
    writer.writerow(['image_id', 'question', 'answer_type', 'expected_answer', 'generated_answer'])
    
    # Write the rows
    for i in range(len(imageId)):
        writer.writerow([imageId[i], questionsStored[i], answerType[i], expectedAnswer[i], results[i]])

print(f"CSV file has been written to {csv_file_path}")

CSV file has been written to /Users/sudhanshu/Desktop/UMASS_COURSES_SEMESTERS/SEM_2/NLP/VITLLMs/src/SegmentedImage_Baseline_VIT.csv


Generating CSV from OG image"

In [27]:

# Function to get answer_type and multiple_choice_answer for a given question_id
def get_answer_details(question_id, data):
    for annotation in data['annotations']:
        if annotation['question_id'] == question_id:
            return annotation['answer_type'], annotation['multiple_choice_answer']
    return None, None

# Replace 'pipe' with the actual function you are using to process images and questions


# Paths to the directories and files
image_directory = "/Users/sudhanshu/Desktop/UMASS_COURSES_SEMESTERS/SEM_2/NLP/VITLLMs/src/dataset/final_eval"
json_file_path = "/Users/sudhanshu/Desktop/UMASS_COURSES_SEMESTERS/SEM_2/NLP/VITLLMs/src/dataset/filtered_dataForTestnew.json"
output_csv_path = "/Users/sudhanshu/Desktop/UMASS_COURSES_SEMESTERS/SEM_2/NLP/VITLLMs/src/Baseline_OriginalImage_Text_Prompts_PreTrainedLLM.csv"

# Load questions from the JSON file
with open(json_file_path, 'r') as json_file:
    data = json.load(json_file)

# Lists to store results
results = []
imageId = []
questionsStored = []
answerType = []
expectedAnswer = []

# Iterate through each annotation in the JSON data
for annotation in data['annotations']:
    image_id = str(annotation['image_id'])  # Convert to string to match with image filename
    question = annotation['question']
    question_id = annotation['question_id']
    answer_type, multiple_choice_answer = get_answer_details(question_id, data)

    # Append the data to respective lists
    imageId.append(image_id)
    questionsStored.append(question)
    answerType.append(answer_type)
    expectedAnswer.append(multiple_choice_answer)

    # Construct the image file name and check if it exists in the directory
    image_name = f"{image_id}.jpg"  # Assuming images are in .jpg format
    image_path = os.path.join(image_directory, image_name)

    if os.path.exists(image_path):
        # Open the image
        image = Image.open(image_path)

        # Process the image with the question
        result = pipe(image, question, top_k=1)
        
        # Append the result to the results list
        results.append(result[0]['answer'])
    else:
        # If the image file does not exist, append a placeholder or handle the case as needed
        results.append("Image not found")

# Write the data to a CSV file
with open(output_csv_path, mode='w', newline='') as file:
    writer = csv.writer(file)
    
    # Write the header
    writer.writerow(['image_id', 'question', 'answer_type', 'expected_answer', 'generated_answer'])
    
    # Write the rows
    for i in range(len(imageId)):
        writer.writerow([imageId[i], questionsStored[i], answerType[i], expectedAnswer[i], results[i]])

print(f"CSV file has been written to {output_csv_path}")



CSV file has been written to /Users/sudhanshu/Desktop/UMASS_COURSES_SEMESTERS/SEM_2/NLP/VITLLMs/src/Baseline_OriginalImage_Text_Prompts_PreTrainedLLM.csv
